In [ ]:
import pandas as pd
import re
import requests
from io import StringIO
from pathlib import Path
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import folium

# Initial step to examine structure and cleanliness of the dataset

In [ ]:
df_test = pd.read_csv("data\\143JourneyDataExtract02Jan2019-08Jan2019.csv")

In [ ]:
df_test.info()

In [ ]:
df_test.head()

## Initial questions and tasks
- convert duration to mins
- convert end date and start date to datetime
- describe duration- avg journey time per bike
- busiest day/time?
- chlropleth map of start and end locations
- how many unique bikes?
- how many journeys per bike/per day?
- how many journeys started and ended at the same location?
- how many journeys over 30 mins?
- above usage trends over the seasons, years, weekdays vs weekends etc

# Download all the relevant files from 2019-21
Summary of cells below
1. Function defined to scrape urls from TFL website
2. Function defined to filter scraped urls by year
3. Function defined to read a single csv from a given url
4. Function defined to read the relevant csvs for a given year and download the data using 10 threads into a parquet file for efficient storage of ~10M records per year
5. Optimise datatypes and combine individual dfs into one and save as parquet for future use

In [ ]:
def scrape_tfl_usage_csvs(
    url: str = "https://cycling.data.tfl.gov.uk/#!usage-stats%2F"
) -> list[str]:
    """
    Scrape TfL cycling usage-stats page using Selenium (handles JavaScript).
    Returns list of JourneyDataExtract CSV URLs.
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(options=options)
    try:
        driver.get(url)
        # Wait for links to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "JourneyDataExtract"))
        )
        
        links = driver.find_elements(By.TAG_NAME, "a")
        urls = []
        for link in links:
            href = link.get_attribute("href")
            if href and "JourneyDataExtract" in href and href.endswith(".csv"):
                urls.append(href)
        
        if not urls:
            raise RuntimeError("No JourneyDataExtract CSVs found")
        
        return sorted(set(urls))
    finally:
        driver.quit()

In [ ]:
def filter_urls_by_year(csv_urls: list[str], year: int) -> list[str]:
    """
    Filter CSV URLs to only include files where the START date matches the given year.
    E.g., Dec2018-Jan2019 belongs to 2018, not 2019.
    
    Args:
        csv_urls: List of CSV URLs from scrape_tfl_usage_csvs()
        year: The year to filter by (e.g., 2019)
        
    Returns:
        List of URLs matching the specified year
    """
    pattern = re.compile(r"JourneyDataExtract(\d{2}[A-Za-z]{3}\d{4})-")
    filtered = []
    
    for url in csv_urls:
        filename = url.split("/")[-1]
        match = pattern.search(filename)
        
        if not match:
            continue
            
        start_date_str = match.group(1)
        try:
            start_date = datetime.strptime(start_date_str, "%d%b%Y")
            if start_date.year == year:
                filtered.append(url)
        except ValueError:
            continue
    
    return filtered

In [ ]:
def download_csv(url: str, headers: dict) -> tuple[str, pd.DataFrame | None]:
    """Download a single CSV and return (filename, dataframe)."""
    filename = url.split("/")[-1]
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text))
        return filename, df
    except Exception as e:
        return filename, None


def download_year_to_parquet_parallel(
    year: int, 
    csv_urls: list[str], 
    output_dir: str = "data",
    max_workers: int = 10
) -> Path:
    """
    Download all CSVs for a given year in parallel and save as a single parquet file.
    
    Args:
        year: The year to download (e.g., 2019)
        csv_urls: List of CSV URLs from scrape_tfl_usage_csvs()
        output_dir: Directory to save the parquet file
        max_workers: Number of parallel downloads (default 10)
        
    Returns:
        Path to the saved parquet file
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # Filter URLs first
    year_urls = filter_urls_by_year(csv_urls, year)
    
    if not year_urls:
        raise ValueError(f"No files found for year {year}")
    
    print(f"Found {len(year_urls)} files for {year}. Downloading with {max_workers} threads...")
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    
    dfs = []
    failed = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_csv, url, headers): url for url in year_urls}
        
        for future in as_completed(futures):
            filename, df = future.result()
            if df is not None:
                dfs.append(df)
                print(f"{filename} appended")
            else:
                failed.append(filename)
                print(f" failed {filename}")
    
    if failed:
        print(f"\nFailed to download {len(failed)} files: {failed}")
    
    if not dfs:
        raise ValueError(f"Failed to download any files for year {year}")
    
    result = pd.concat(dfs, ignore_index=True)
    
    # Convert date columns if they exist
    date_cols = ["Start Date", "End Date"]
    for col in date_cols:
        if col in result.columns:
            result[col] = pd.to_datetime(result[col], dayfirst=True)
    
    parquet_path = output_path / f"journeys_{year}.parquet"
    result.to_parquet(parquet_path, index=False)
    print(f"\nSaved {len(result):,} rows from {len(dfs)} files to {parquet_path}")
    
    return parquet_path

In [ ]:
# First get all the CSV URLs
csv_urls = scrape_tfl_usage_csvs()

In [ ]:
# Then download a specific year

# download_year_to_parquet(2019, csv_urls) # (single thread took 4m 20.2s)
# download_year_to_parquet(2020, csv_urls) # (single-thread took 5m 53.7s)
# download_year_to_parquet_parallel(2021, csv_urls)

In [ ]:
# # Combine all years into one parquet
df_2019 = pd.read_parquet("data/journeys_2019.parquet")
df_2020 = pd.read_parquet("data/journeys_2020.parquet")
df_2021 = pd.read_parquet("data/journeys_2021.parquet")

df = pd.concat([df_2019, df_2020, df_2021], ignore_index=True)

print(f"Combined: {len(df):,} rows")
print(f"2019: {len(df_2019):,} | 2020: {len(df_2020):,} | 2021: {len(df_2021):,}")

In [ ]:
# Optimise memory usage

# df = df.drop(columns=['Rental Id'])
# # remove records where date is 2022
# df = df[df['Start Date'].dt.year != 2022]

# # convert duration to mins
# df['Duration'] = (df['Duration'] / 60).astype('float32')

# # add DoW column
# df['DayOfWeek'] = df['Start Date'].dt.day_name().astype('category')

# df['Date'] = (df['Start Date'].dt.date).astype('datetime64[ns]')
# df['Year'] = (df['Start Date'].dt.year).astype('category') # only 3 years so category is fine

# df['Start Hour'] = (df['Start Date'].dt.hour).astype('uint8')

# # # Convert station names to category (big savings for repeated strings)
# df['EndStation Name'] = df['EndStation Name'].astype('category')
# df['StartStation Name'] = df['StartStation Name'].astype('category')
# df['Bike Id'] = df['Bike Id'].astype('category')
# df['EndStation Id'] = df['EndStation Id'].astype('category')
# df['StartStation Id'] = df['StartStation Id'].astype('category')

# df.info(memory_usage='deep')

In [ ]:
# save combined df as parquet for easier read
# df.to_parquet("data/journeys_2019_2020_2021.parquet", engine="pyarrow", index=False)

# EDA on full dataset
Summary
1. Remove records where date not within 2019-2021
2. Prove non-normality of data
3. Remove outliers based on threshold of 24 hours
4. EDA

## Load data

In [ ]:
df = pd.read_parquet("data/journeys_2019_2020_2021.parquet")
cat_cols = ['Bike Id', 'EndStation Id', 'StartStation Id', 'Year']
for c in cat_cols:
    df[c] = df[c].astype('category')

In [ ]:
df.info()

In [ ]:
df.sample(20)

In [ ]:
df.tail()

In [ ]:
# are there any missing values?
df[['Start Date', 'End Date', 'Duration', 'Bike Id', 'StartStation Id', 'EndStation Id']].isnull().sum()

## Is the data normally distributed?
Answer: no, find alternatives to z score outlier removal

In [ ]:
# 31M rows is too large a sample for plotting, so take a random sample of 1M rows

# Take stratified random sample across years
df['Year'] = df['Start Date'].dt.year
sample = df.groupby('Year', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 100_000), random_state=42)
)
print(f"Sample size: {len(sample):,} rows")
print(f"Sample by year:\n{sample['Year'].value_counts().sort_index()}")

# Skewness and Kurtosis
skewness = sample['Duration'].skew()
kurtosis = sample['Duration'].kurtosis()

print(f"\n--- Normality Indicators ---")
print(f"Skewness: {skewness:.2f}  (normal ≈ 0)")
print(f"Kurtosis: {kurtosis:.2f}  (normal ≈ 0, scipy uses excess kurtosis)")

# Interpretation
if abs(skewness) < 0.5:
    print("-- Skewness: Approximately symmetric")
elif skewness > 0:
    print("-- Skewness: Right-skewed (long tail of high values)")
else:
    print("-- Skewness: Left-skewed (long tail of low values)")

if abs(kurtosis) < 1:
    print("-- Kurtosis: Similar to normal distribution")
elif kurtosis > 0:
    print("-- Kurtosis: Heavy tails (more outliers than normal)")
else:
    print("-- Kurtosis: Light tails (fewer outliers than normal)")

# D'Agostino-Pearson test (on smaller sample for speed)
test_sample = sample['Duration'].sample(n=5000, random_state=42)
stat, p_value = stats.normaltest(test_sample)
print(f"\nD'Agostino-Pearson test: statistic={stat:.2f}, p-value={p_value:.2e}")
print(f"-- {'NOT normally distributed (p < 0.05)' if p_value < 0.05 else 'Could be normal (p >= 0.05)'}")

In [ ]:
# histogram with KDE of the sample data
plt.figure(figsize=(10, 6))
sns.histplot(sample['Duration'], bins=1000, kde=True, stat='density', color='skyblue')
plt.xlim(0, sample['Duration'].quantile(0.99))  # limit x-axis to 99th percentile for clarity
plt.title('Histogram and KDE of Journey Durations (Sample)')

In [ ]:
# QQ plot
plt.figure(figsize=(6, 6))
stats.probplot(sample['Duration'], dist="norm", plot=plt)
plt.title('QQ Plot of Journey Durations (Sample)')


This plot suggests most of the data is concentrated at the lower values with some extremely high values, explaining the steep rise on the right. With the histogram and the QQ plot, we can conclude that the data is non-normal. This makes sense given the large variety of individual bike rides and variable and unquantifiable traffic scenarios in this dataset.

In [ ]:
df['Duration'].describe()

## Outliers
- Since the data is not normally distributed, use a reasonable value as a threshold. In this case, assume rentals <24h are valid.
- Also the client is interested in short term rentals, so long term rentals are outliers for this use case. Drop these values from further analysis

In [ ]:
# describe duration- avg journey time per bike
df['Duration'].describe()

In [ ]:
# outliers in duration? if so where and which bikes? what times?
# since the data is not normally distributed, use a reasonable value as a threshold. In this case, assume rentals <24h are valid.
# Also the client is interested in short term rentals, so long term rentals are outliers.

outliers = df[df['Duration'] >= 1440].sort_values(by='Duration', ascending=False) # 9258 records
outliers
# count by day of week
# outliers['DayOfWeek'].value_counts()

## EDA

In [ ]:
df_no_outliers = df[~df.index.isin(outliers.index)].copy()
df_no_outliers.info()

In [ ]:
df_no_outliers.head()

### Bike
1. Number of unique bikes - 17766
2. Top 10 most used bike ids - [16151, 16082, 15331, 16011, 15529, 16045, 16379, 16344, 15337]
3. How many journeys per bike/per day in each of the years?

In [ ]:
# are there bike ids with a lot of usage?
print (df_no_outliers['Bike Id'].value_counts().head(10))
df_no_outliers['Bike Id'].value_counts().describe()  # 17766 unique bike ids

In [ ]:
# how many journeys per bike/per day in each of the years?
# Calculate journeys per bike per day
# df_no_outliers['Date'] = (df_no_outliers['Start Date'].dt.date).astype('datetime64[ns]')
# df_no_outliers['Year'] = (df_no_outliers['Start Date'].dt.year).astype('uint8')

journeys_per_bike_per_day = df_no_outliers.groupby(
    ['Year', 'Bike Id', 'Date'], observed=True
).size().reset_index(name='journeys')

# Summary stats by year
summary = journeys_per_bike_per_day.groupby('Year')['journeys'].agg(
    ['mean', 'median', 'std', 'min', 'max', 'count']
).round(2)
summary.columns = ['Avg Journeys/Bike/Day', 'Median', 'Std Dev', 'Min', 'Max', 'Total Bike-Days']
print("Journeys per bike per day by year:")
print(summary)

# Distribution visualization
fig = go.Figure()

for year in sorted(journeys_per_bike_per_day['Year'].unique()):
    year_data = journeys_per_bike_per_day[journeys_per_bike_per_day['Year'] == year]['journeys']
    fig.add_trace(go.Box(
        y=year_data,
        name=str(year),
        boxmean=True  # shows mean as dashed line
    ))

fig.update_layout(
    title='<b>Distribution of Daily Journeys per Bike by Year</b>',
    yaxis_title='Journeys per Bike per Day',
    xaxis_title='Year',
    height=500,
    width=700
)
fig.show()
# fig.write_html("journeys_per_bike_per_day_by_year.html")

### Time
1. Busiest day- Saturday, but not much variation between other days, maximum 11%
2. Busiest times- 7-9am during morning rush, 3-8pm during afternoon/evening rush. This indicates a clear commuter pattern. Bikes need to be redistributed before 7am and low overnight usage is an opportunity for maintenance window.

In [ ]:
# busiest day/time?
df_no_outliers['DayOfWeek'].value_counts(ascending=False)

Above indicates fairly consistent use of bikes on all days of the week, with only a 11% variation on the busiest and quietest days

In [ ]:
# # Are there any time of day patterns?

# # Extract hour from start and end times
# df_no_outliers['Start Hour'] = df_no_outliers['Start Date'].dt.hour
# df_no_outliers['End Hour'] = df_no_outliers['End Date'].dt.hour

# # Count journeys by hour
# start_counts = df_no_outliers['Start Hour'].value_counts().sort_index().reset_index()
# start_counts.columns = ['hour', 'count']

# end_counts = df_no_outliers['End Hour'].value_counts().sort_index().reset_index()
# end_counts.columns = ['hour', 'count']

# # Define hour mapping for clock face
# hour_mapping = {0: 0, 1: 30, 2: 60, 3: 90, 4: 120, 5: 150, 6: 180,
#                 7: 210, 8: 240, 9: 270, 10: 300, 11: 330, 12: 0,
#                 13: 30, 14: 60, 15: 90, 16: 120, 17: 150, 18: 180,
#                 19: 210, 20: 240, 21: 270, 22: 300, 23: 330}

# start_counts['theta'] = start_counts['hour'].map(hour_mapping)
# end_counts['theta'] = end_counts['hour'].map(hour_mapping)

# # Colors for AM/PM
# color_map = {
#     "start_am": "#88D8B0",  # Green
#     "start_pm": "#40B8E1",  # Blue
#     "end_am": "#F271A7",    # Pink
#     "end_pm": "#FF9F80"     # Orange
# }

# fig = make_subplots(
#     rows=1, cols=2,
#     specs=[[{'type': 'polar'}, {'type': 'polar'}]],
#     subplot_titles=["Journey Starts", "Journey Ends"]
# )

# # Journey Starts - AM & PM
# for is_am in [True, False]:
#     filtered = start_counts[(start_counts['hour'] < 12) if is_am else (start_counts['hour'] >= 12)]
#     fig.add_trace(
#         go.Barpolar(
#             r=filtered['count'],
#             theta=filtered['theta'],
#             width=30,
#             name="AM" if is_am else "PM",
#             marker_color=color_map["start_am"] if is_am else color_map["start_pm"],
#             text=[f"{h}:00 - {c:,} trips" for h, c in zip(filtered['hour'], filtered['count'])],
#             hoverinfo="text"
#         ),
#         row=1, col=1
#     )

# # Journey Ends - AM & PM
# for is_am in [True, False]:
#     filtered = end_counts[(end_counts['hour'] < 12) if is_am else (end_counts['hour'] >= 12)]
#     fig.add_trace(
#         go.Barpolar(
#             r=filtered['count'],
#             theta=filtered['theta'],
#             width=30,
#             name="AM" if is_am else "PM",
#             marker_color=color_map["end_am"] if is_am else color_map["end_pm"],
#             text=[f"{h}:00 - {c:,} trips" for h, c in zip(filtered['hour'], filtered['count'])],
#             hoverinfo="text"
#         ),
#         row=1, col=2
#     )

# # Layout
# polar_axis = dict(
#     radialaxis=dict(visible=False),
#     angularaxis=dict(
#         tickmode="array",
#         tickvals=[0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330],
#         ticktext=["12", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"],
#         direction="clockwise",
#         rotation=90
#     )
# )

# fig.update_layout(
#     height=600,
#     width=1000,
#     title="Hourly Bike Usage Patterns",
#     polar=polar_axis,
#     polar2=polar_axis,
#     showlegend=False
# )

# # Custom legend
# legend_items = [
#     ("Starts AM", color_map["start_am"]),
#     ("Starts PM", color_map["start_pm"]),
#     ("Ends AM", color_map["end_am"]),
#     ("Ends PM", color_map["end_pm"]),
# ]
# for idx, (label, color) in enumerate(legend_items):
#     fig.add_annotation(
#         x=1.08, y=0.9 - (idx * 0.08),
#         xref='paper', yref='paper',
#         text=f"<b>{label}</b>",
#         showarrow=False,
#         font=dict(color=color, size=12)
#     )

# fig.show()

In [ ]:
# fig.write_html("hourly bike usage.html")

The most number of journeys in the morning are between 7-9am and 4-8pm indicating a clear commuter pattern. Bikes need to be redistributed before 7am and low overnight usage is an opportunity for maintenance window.

In [ ]:
# Count journeys by start hour
# df_no_outliers['Start Hour'] = df_no_outliers['Start Date'].dt.hour
hourly = df_no_outliers.groupby('Start Hour').size().reset_index()
hourly.columns = ['hour', 'count']

# Define rush hours
rush_hours = list(range(7, 10)) + list(range(15, 21))  # 7-9AM and 3-8PM
hourly['is_rush'] = hourly['hour'].isin(rush_hours)
hourly['color'] = hourly['is_rush'].map({True: '#2C3E50', False: '#88D8B0'})

# Format labels (show as millions with 1 decimal)
hourly['label'] = hourly['count'].apply(lambda x: f'{x/1e6:.2f}M')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=hourly['hour'],
    y=hourly['count'],
    marker_color=hourly['color'],
    text=hourly['label'],
    textposition='outside',
    textangle=-25,
    textfont=dict(size=10),
    hovertemplate='%{x}:00<br>%{y:,.0f} journeys<extra></extra>'
))

# Layout
fig.update_layout(
    title=dict(
        text='<b>When Do Londoners Use Bikes?</b><br><sup>Rush hours highlighted (7-9AM & 3-8PM)</sup>',
        x=0.5
    ),
    xaxis=dict(
        title='Hour of Day',
        tickmode='array',
        tickvals=list(range(24)),
        ticktext=[f'{h:02d}:00' for h in range(24)],
        tickangle=-45
    ),
    yaxis=dict(
        title='Number of Journeys',
        tickformat=',.0f'
    ),
    height=650,
    width=1000,
    showlegend=False,
    margin=dict(t=100, b=80),
    
    # Add annotation for insight
    annotations=[
        dict(
            x=0.01, y=0.98,
            xref='paper', yref='paper',
            text='<b>Key Insight:</b><br>Peak at 8AM & 6PM<br>→ Strong commuter pattern',
            showarrow=False,
            align='left',
            bgcolor='rgba(255,248,220,0.8)',
            bordercolor='#888',
            borderwidth=1,
            borderpad=6,
            font=dict(size=11)
        )
    ]
)

fig.show()
# fig.write_html("hourly_usage_bars.html")

In [ ]:
df_no_outliers.info()

### Duration/usage

In [ ]:
# average journey duration by start and end stations
avg_duration_start = df_no_outliers.groupby('StartStation Name', observed=True)['Duration'].mean().sort_values(ascending=False).head(10)
avg_duration_end = df_no_outliers.groupby('EndStation Name', observed=True)['Duration'].mean().sort_values(ascending=False).head(10)
print("Top 10 Start Stations by Average Duration:")
print(avg_duration_start)
print("Top 10 End Stations by Average Duration:")
print(avg_duration_end)

In [ ]:
# how many journeys over and under 30 mins?
under_30mins = df_no_outliers[df_no_outliers['Duration'] <= 30]
over_30mins = df_no_outliers[(df_no_outliers['Duration'] > 30) & (df_no_outliers['Duration'] < 60)] # but under 60 mins
print(f"Journeys <= 30 mins: {len(under_30mins):,} ({len(under_30mins) / len(df_no_outliers) * 100:.2f}%)")
print(f"Journeys > 30 mins <=60 mins: {len(over_30mins):,} ({len(over_30mins) / len(df_no_outliers) * 100:.2f}%)")
# 4.63% are over 60 mins

In [ ]:
# weekday vs weekend duration and number of journeys
# Create weekend flag
df_no_outliers['IsWeekend'] = df_no_outliers['DayOfWeek'].isin(['Saturday', 'Sunday'])

# Calculate AVERAGE journeys per day (normalized metric)
daily_counts = df_no_outliers.groupby(['Date', 'IsWeekend']).size().reset_index(name='journeys')
avg_per_day = daily_counts.groupby('IsWeekend')['journeys'].mean().reset_index()
avg_per_day['Day Type'] = avg_per_day['IsWeekend'].map({False: 'Weekday', True: 'Weekend'})

# Calculate average duration
avg_duration = df_no_outliers.groupby('IsWeekend')['Duration'].mean().reset_index()
avg_duration['Day Type'] = avg_duration['IsWeekend'].map({False: 'Weekday', True: 'Weekend'})

print("=== Weekday vs Weekend Summary ===")
print(f"\nAverage Journeys Per Day:")
for _, row in avg_per_day.iterrows():
    print(f"  {row['Day Type']}: {row['journeys']:,.0f}")

print(f"\nAverage Duration (mins):")
for _, row in avg_duration.iterrows():
    print(f"  {row['Day Type']}: {row['Duration']:.1f}")

# Create side-by-side visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['<b>Avg Journeys per Day</b>', '<b>Avg Duration (mins)</b>']
)

colors = ['#2C3E50', '#88D8B0']  # Weekday dark, Weekend light

# Bar 1: Average journeys per day
fig.add_trace(
    go.Bar(
        x=avg_per_day['Day Type'],
        y=avg_per_day['journeys'],
        marker_color=colors,
        text=avg_per_day['journeys'].apply(lambda x: f'{x:,.0f}'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=1
)

# Bar 2: Average duration
fig.add_trace(
    go.Bar(
        x=avg_duration['Day Type'],
        y=avg_duration['Duration'],
        marker_color=colors,
        text=avg_duration['Duration'].apply(lambda x: f'{x:.1f}'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_layout(
    title='<b>Weekday vs Weekend Usage Patterns</b>',
    height=450,
    width=800,
    margin=dict(t=80)
)

fig.update_yaxes(title_text='Journeys', row=1, col=1)
fig.update_yaxes(title_text='Minutes', row=1, col=2)

fig.show()
# fig.write_html("weekday_vs_weekend_summary.html")

In [ ]:
# Seasonal Analysis
# Define seasons based on month
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df_no_outliers['Month'] = df_no_outliers['Start Date'].dt.month
df_no_outliers['Season'] = df_no_outliers['Month'].apply(get_season)

# Calculate average journeys per day by season
daily_by_season = df_no_outliers.groupby(['Date', 'Season']).size().reset_index(name='journeys')
avg_journeys = daily_by_season.groupby('Season')['journeys'].mean().reset_index()

# Calculate average duration by season
avg_duration = df_no_outliers.groupby('Season')['Duration'].mean().reset_index()

# Merge for plotting
seasonal = avg_journeys.merge(avg_duration, on='Season')
seasonal.columns = ['Season', 'Avg Journeys/Day', 'Avg Duration (mins)']

# Order seasons logically
season_order = ['Spring', 'Summer', 'Autumn', 'Winter']
seasonal['Season'] = pd.Categorical(seasonal['Season'], categories=season_order, ordered=True)
seasonal = seasonal.sort_values('Season')

print("=== Seasonal Summary ===")
print(seasonal.to_string(index=False))

# Create visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['<b>Avg Journeys per Day</b>', '<b>Avg Duration (mins)</b>']
)

colors = ['#88D8B0', '#FF6B6B', '#FFE66D', '#4ECDC4']  # Spring, Summer, Autumn, Winter

fig.add_trace(
    go.Bar(
        x=seasonal['Season'],
        y=seasonal['Avg Journeys/Day'],
        marker_color=colors,
        text=seasonal['Avg Journeys/Day'].apply(lambda x: f'{x:,.0f}'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=seasonal['Season'],
        y=seasonal['Avg Duration (mins)'],
        marker_color=colors,
        text=seasonal['Avg Duration (mins)'].apply(lambda x: f'{x:.1f}'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_layout(
    title='<b>Seasonal Usage Patterns</b>',
    height=450,
    width=900,
    margin=dict(t=80)
)

fig.update_yaxes(title_text='Journeys/Day', row=1, col=1)
fig.update_yaxes(title_text='Minutes', row=1, col=2)

fig.show()
# fig.write_html("seasonal_usage_summary.html")

In [ ]:
# Yearly Analysis - Avg Journeys/Day and Avg Duration
# Calculate average journeys per day by year
daily_by_year = df_no_outliers.groupby(['Date', 'Year']).size().reset_index(name='journeys')
avg_journeys_year = daily_by_year.groupby('Year')['journeys'].mean().reset_index()

# Calculate average duration by year
avg_duration_year = df_no_outliers.groupby('Year')['Duration'].mean().reset_index()

# Merge for plotting
yearly = avg_journeys_year.merge(avg_duration_year, on='Year')
yearly.columns = ['Year', 'Avg Journeys/Day', 'Avg Duration (mins)']

print("=== Yearly Summary ===")
print(yearly.to_string(index=False))

# Create visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['<b>Avg Journeys per Day</b>', '<b>Avg Duration (mins)</b>']
)

colors = ['#2C3E50', '#E74C3C', '#3498DB']  # 2019, 2020, 2021

fig.add_trace(
    go.Bar(
        x=yearly['Year'].astype(str),
        y=yearly['Avg Journeys/Day'],
        marker_color=colors,
        text=yearly['Avg Journeys/Day'].apply(lambda x: f'{x:,.0f}'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=yearly['Year'].astype(str),
        y=yearly['Avg Duration (mins)'],
        marker_color=colors,
        text=yearly['Avg Duration (mins)'].apply(lambda x: f'{x:.1f}'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_layout(
    title='<b>Yearly Usage Patterns (2019-2021)</b>',
    height=450,
    width=800,
    margin=dict(t=80)
)

fig.update_yaxes(title_text='Journeys/Day', row=1, col=1)
fig.update_yaxes(title_text='Minutes', row=1, col=2)

fig.show()
# fig.write_html("yearly_usage_summary.html")

2020: Likely lower journeys/day (COVID lockdowns), but possibly longer durations (leisure rides during exercise allowance, fewer short commutes) but surprisingly there was only a 0.64% decrease from 2019.
2021: Recovery- 5.5% increase since 2020
Longer durations in 2020-21 may indicate shift from commuting to recreational use

In [ ]:
# df_no_outliers.drop(columns=['IsWeekend', 'Month', 'Season'], inplace=True)
df_no_outliers.info()

### Location analyses
- chlropleth map of start and end locations
- how many journeys started and ended at the same location?
- most popular start and end stations?

In [ ]:
df_no_outliers['StartStation Id'].nunique(), df_no_outliers['EndStation Id'].nunique()

In [ ]:
# Step 1: Get station coordinates from TfL API

# TfL BikePoint API (free, no key needed)
url = "https://api.tfl.gov.uk/BikePoint"
response = requests.get(url)
stations = response.json()

# Create lookup: station_id -> (lat, lon, name)
station_coords = {}
for s in stations:
    # Extract ID number from "BikePoints_123" format
    station_id = int(s['id'].replace('BikePoints_', ''))
    station_coords[station_id] = {
        'lat': s['lat'],
        'lon': s['lon'],
        'name': s['commonName']
    }

print(f"Loaded {len(station_coords)} stations with coordinates")

stations_df = (
    pd.DataFrame.from_dict(station_coords, orient='index')
    .rename_axis('StationId')
    .reset_index()
    [['StationId', 'lat', 'lon']]
)

stations_df['lat'] = stations_df['lat'].astype('float32')
stations_df['lon'] = stations_df['lon'].astype('float32')
stations_df.info()

In [ ]:
# store stations_df for future use
stations_df.to_parquet("data/station_coordinates.parquet", engine="pyarrow", index=False)


In [ ]:
stations = pd.read_parquet("data/station_coordinates.parquet")
stations.info()
stations.head()

In [ ]:
stations['StationId'].describe()

In [ ]:
df_no_outliers.head()

### Station imbalance 
1. Which stations are are net sources vs net sinks of bikes - critical for operational planning:
- Red circles = More journeys END here (bikes accumulate)
- Green circles = More journeys START here (bikes get depleted)
- Size = magnitude of imbalance
- This directly addresses supply chain management concerns.

In [ ]:
# Calculate station imbalance (ends - starts)
starts = df_no_outliers.groupby('StartStation Id', observed=True).size()
ends = df_no_outliers.groupby('EndStation Id', observed=True).size()

# Merge to get both metrics
station_balance = pd.DataFrame({
    'starts': starts,
    'ends': ends
}).fillna(0).astype(int)

station_balance['net_imbalance'] = station_balance['ends'] - station_balance['starts']
station_balance['total_volume'] = station_balance['starts'] + station_balance['ends']
station_balance['imbalance_pct'] = (station_balance['net_imbalance'] / station_balance['total_volume'] * 100).round(1)

# Add station names
station_names = df_no_outliers.groupby('StartStation Id', observed=True)['StartStation Name'].first()
station_balance = station_balance.merge(
    station_names.rename('name'), 
    left_index=True, 
    right_index=True, 
    how='left'
)

print(f"Total stations: {len(station_balance)}")
print(f"\nTop 5 Sinks (bikes accumulate):")
print(station_balance.nlargest(5, 'net_imbalance')[['name', 'starts', 'ends', 'net_imbalance', 'imbalance_pct']])
print(f"\nTop 5 Sources (bikes depleted):")
print(station_balance.nsmallest(5, 'net_imbalance')[['name', 'starts', 'ends', 'net_imbalance', 'imbalance_pct']])

# Visualization: Horizontal Bar Chart
top_sinks = station_balance.nlargest(10, 'net_imbalance').sort_values('net_imbalance')
top_sources = station_balance.nsmallest(10, 'net_imbalance').sort_values('net_imbalance', ascending=False)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['<b>Top 10 Sources</b><br>(Bikes Depleted)', '<b>Top 10 Sinks</b><br>(Bikes Accumulate)'],
    horizontal_spacing=0.25
)

# Sources (left, negative values)
fig.add_trace(
    go.Bar(
        y=top_sources['name'],
        x=top_sources['net_imbalance'],
        orientation='h',
        marker_color='#E74C3C',
        text=top_sources['net_imbalance'].apply(lambda x: f'{x:,}'),
        textposition='outside',
        textfont=dict(size=10),
        hovertemplate='<b>%{y}</b><br>Net: %{x:,} bikes<br>Starts: ' + 
                      top_sources['starts'].astype(str) + '<br>Ends: ' + 
                      top_sources['ends'].astype(str) + '<extra></extra>',
        showlegend=False
    ),
    row=1, col=1
)

# Sinks (right, positive values)
fig.add_trace(
    go.Bar(
        y=top_sinks['name'],
        x=top_sinks['net_imbalance'],
        orientation='h',
        marker_color='#27AE60',
        text=top_sinks['net_imbalance'].apply(lambda x: f'+{x:,}'),
        textposition='outside',
        textfont=dict(size=10),
        hovertemplate='<b>%{y}</b><br>Net: +%{x:,} bikes<br>Starts: ' + 
                      top_sinks['starts'].astype(str) + '<br>Ends: ' + 
                      top_sinks['ends'].astype(str) + '<extra></extra>',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_layout(
    title=dict(
        text='<b>Station Imbalance: Where to Redistribute Bikes</b><br><sup>Based on 31.4M journeys (2019-2021)</sup>',
        x=0.5
    ),
    height=700,
    width=1500,
    margin=dict(t=50, l=200, r=50)
)

fig.update_xaxes(title_text='Net Imbalance (bikes)', row=1, col=1)
fig.update_xaxes(title_text='Net Imbalance (bikes)', row=1, col=2)

fig.add_annotation(
    x=0.5, y=-0.12,
    xref='paper', yref='paper',
    text='<b>Action:</b> Sources need morning restocking | Sinks need evening collection',
    showarrow=False,
    bgcolor='rgba(255,248,220,0.8)',
    bordercolor='#888',
    borderwidth=1,
    borderpad=6,
    font=dict(size=11)
)

fig.show()
fig.write_html("station_imbalance_bars.html")

In [ ]:
# Merge station_balance with coordinates

station_balance_geo = station_balance.reset_index().merge(
    stations,
    left_index=True,
    right_on='StationId',
    how='inner'
)

# Create map centered on London
london_map = folium.Map(
    location=[51.5074, -0.1278],
    zoom_start=12,
    tiles='cartodbpositron'
)

# Add markers for top 20 imbalanced stations (10 sources + 10 sinks)
top_20_imbalanced = pd.concat([
    station_balance_geo.nlargest(10, 'net_imbalance'),
    station_balance_geo.nsmallest(10, 'net_imbalance')
])

for _, row in top_20_imbalanced.iterrows():
    is_sink = row['net_imbalance'] > 0
    color = 'green' if is_sink else 'red'
    icon_symbol = 'arrow-up' if is_sink else 'arrow-down'
    
    # Marker size based on magnitude
    radius = min(abs(row['net_imbalance']) / 100, 20)  # Cap at 20
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=folium.Popup(
            f"""<b>{row['name']}</b><br>
            Net: {row['net_imbalance']:+,} bikes<br>
            Starts: {row['starts']:,}<br>
            Ends: {row['ends']:,}<br>
            Type: {'SINK (accumulates)' if is_sink else 'SOURCE (depleted)'}""",
            max_width=250
        ),
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6,
        weight=2
    ).add_to(london_map)

# Add legend
legend_html = '''
<div style="position: fixed; 
     top: 10px; right: 10px; width: 180px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <p><b>Station Imbalance</b></p>
     <p><span style="color:green;">●</span> Sinks (bikes accumulate)</p>
     <p><span style="color:red;">●</span> Sources (bikes depleted)</p>
     <p style="font-size:11px; margin-top:10px;">Circle size = magnitude</p>
</div>
'''
london_map.get_root().html.add_child(folium.Element(legend_html))

london_map.save('station_imbalance_map.html')
print("Map saved to station_imbalance_map.html")
london_map

In [ ]:
# Merge station_balance (which has total_volume) with coordinates
station_popularity_geo = station_balance_geo.copy()
# station_popularity_geo = station_balance.merge(
#     stations,
#     left_index=True,
#     right_on='StationId',
#     how='inner'
# )

# Create map centered on London
popularity_map = folium.Map(
    location=[51.5074, -0.1278],
    zoom_start=12,
    tiles='cartodbpositron'
)

# Normalize radius for visualization (scale to reasonable circle sizes)
max_volume = station_popularity_geo['total_volume'].max()
station_popularity_geo['radius'] = (station_popularity_geo['total_volume'] / max_volume * 25) + 3  # 3-28 range

# Add all stations, colored by volume intensity
for _, row in station_popularity_geo.iterrows():
    # Color intensity based on volume (blue gradient)
    intensity = row['total_volume'] / max_volume
    color = f'#{int(44 + (1-intensity)*100):02x}{int(62 + (1-intensity)*100):02x}{int(80 + (1-intensity)*80):02x}'
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['radius'],
        popup=folium.Popup(
            f"""<b>{row['name']}</b><br>
            Total Journeys: {row['total_volume']:,}<br>
            Starts: {row['starts']:,}<br>
            Ends: {row['ends']:,}""",
            max_width=250
        ),
        color='#2C3E50',
        fill=True,
        fillColor='#3498DB',
        fillOpacity=0.4 + (intensity * 0.4),  # 0.4-0.8 opacity
        weight=1
    ).add_to(popularity_map)

# Highlight top 20 busiest stations with labels
top_10_popular = station_popularity_geo.nlargest(20, 'total_volume')
for _, row in top_10_popular.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['radius'],
        popup=folium.Popup(
            f"""<b>🔥 {row['name']}</b><br>
            Total Journeys: {row['total_volume']:,}<br>
            Rank: Top 20 Busiest""",
            max_width=250
        ),
        color='#E74C3C',
        fill=True,
        fillColor='#E74C3C',
        fillOpacity=0.7,
        weight=3
    ).add_to(popularity_map)

# Add legend
legend_html = '''
<div style="position: fixed; 
     top: 10px; right: 10px; width: 200px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <p><b>Station Popularity</b></p>
     <p><span style="color:#E74C3C;">●</span> Top 10 Busiest</p>
     <p><span style="color:#3498DB;">●</span> All Stations</p>
     <p style="font-size:11px; margin-top:10px;">Circle size = total journeys<br>(starts + ends)</p>
</div>
'''
popularity_map.get_root().html.add_child(folium.Element(legend_html))

popularity_map.save('station_popularity_map.html')
print("Map saved to station_popularity_map.html")

# Print top 10 summary
print("\n=== Top 20 Busiest Stations ===")
print(top_10_popular[['name', 'total_volume', 'starts', 'ends']].to_string(index=False))

popularity_map

In [ ]:
# Find journeys where start and end station are the same
same_station = df_no_outliers[
    df_no_outliers['StartStation Id'] == df_no_outliers['EndStation Id']
].copy()

total_journeys = len(df_no_outliers)
same_station_count = len(same_station)

print(f"=== Same-Station Journeys (Round Trips) ===")
print(f"Total journeys: {total_journeys:,}")
print(f"Same-station journeys: {same_station_count:,} ({same_station_count/total_journeys*100:.2f}%)")
print(f"Different-station journeys: {total_journeys - same_station_count:,} ({(total_journeys - same_station_count)/total_journeys*100:.2f}%)")

# Average duration comparison
print(f"\n=== Duration Comparison ===")
print(f"Same-station avg duration: {same_station['Duration'].mean():.1f} mins")
print(f"Different-station avg duration: {df_no_outliers[df_no_outliers['StartStation Id'] != df_no_outliers['EndStation Id']]['Duration'].mean():.1f} mins")

# Top stations for same-station journeys (likely leisure/parks)
same_station_by_station = same_station.groupby(
    ['StartStation Id', 'StartStation Name'], observed=True
).agg(
    round_trips=('Duration', 'count'),
    avg_duration=('Duration', 'mean')
).reset_index()
same_station_by_station.columns = ['StationId', 'name', 'round_trips', 'avg_duration']
same_station_by_station = same_station_by_station.sort_values('round_trips', ascending=False)

print(f"\n=== Top 10 Stations for Round Trips ===")
print(same_station_by_station.head(10)[['name', 'round_trips', 'avg_duration']].to_string(index=False))

# Visualization: Bar chart
top_round_trip_stations = same_station_by_station.head(15)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_round_trip_stations['name'],
    x=top_round_trip_stations['round_trips'],
    orientation='h',
    marker_color='#9B59B6',
    text=top_round_trip_stations['round_trips'].apply(lambda x: f'{x:,}'),
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Round trips: %{x:,}<extra></extra>'
))

fig.update_layout(
    title=dict(
        text=f'<b>Top 15 Stations for Round Trips</b><br><sup>{same_station_count:,} same-station journeys ({same_station_count/total_journeys*100:.1f}% of total)</sup>',
        x=0.5
    ),
    xaxis_title='Number of Round Trips',
    yaxis=dict(autorange='reversed'),
    height=600,
    width=900,
    margin=dict(l=250)
)

fig.show()
# fig.write_html("same_station_journeys.html")

# Map: Highlight round-trip hotspots
round_trip_geo = same_station_by_station.merge(
    stations,
    on='StationId',
    how='inner'
)

round_trip_map = folium.Map(
    location=[51.5074, -0.1278],
    zoom_start=12,
    tiles='cartodbpositron'
)

# Normalize for circle size
max_trips = round_trip_geo['round_trips'].max()

for _, row in round_trip_geo.nlargest(30, 'round_trips').iterrows():
    radius = (row['round_trips'] / max_trips * 20) + 5
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=folium.Popup(
            f"""<b>{row['name']}</b><br>
            Round Trips: {row['round_trips']:,}<br>
            Avg Duration: {row['avg_duration']:.1f} mins<br>
            <i>Likely leisure/park location</i>""",
            max_width=250
        ),
        color='#9B59B6',
        fill=True,
        fillColor='#9B59B6',
        fillOpacity=0.6,
        weight=2
    ).add_to(round_trip_map)

# Legend
legend_html = '''
<div style="position: fixed; 
     top: 10px; right: 10px; width: 200px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
     <p><b>Round Trip Hotspots</b></p>
     <p><span style="color:#9B59B6;">●</span> Same start & end station</p>
     <p style="font-size:11px; margin-top:10px;">Circle size = # round trips<br>
     Likely parks or leisure areas</p>
</div>
'''
round_trip_map.get_root().html.add_child(folium.Element(legend_html))

round_trip_map.save('round_trip_hotspots_map.html')
print("\nMap saved to round_trip_hotspots_map.html")
round_trip_map